In [2]:
# import necessary libraries
import csv

In [27]:
with open('weather2016_1.csv', 'w', newline='') as outputfile:
    writer = csv.writer(outputfile, delimiter=',')
    with open('weather2016.csv') as inputfile:
        reader = csv.reader(inputfile)
        for row in reader:
            # replace the '-' and 'Trace' in Rainfall with 0
            if row[11] == '-' or row [11] == 'Trace':
                row[11] = '0'
            # replace the '-' in Sunshine with 0
            if row[13] == '-':
                row[13] = '0'
            
            writer.writerow(row)

In [28]:
with open('weather2016_2.csv', 'w', newline='') as outputfile:
    writer = csv.writer(outputfile, delimiter=',')
    with open('weather2016_1.csv') as inputfile:
        reader = csv.reader(inputfile)
        prev_row = None
        for row in reader:
            if row[15] == 'N.A.' or row[15] == 'N/A':
                row[15] = ''
            prev_row = row
            writer.writerow(row)

In [29]:
with open('weather2017_3.csv', 'w', newline='') as outputfile:
    writer = csv.writer(outputfile, delimiter=',')
    with open('weather2017_2.csv') as inputfile:
        reader = csv.reader(inputfile)
        first_row = True
        for row in reader:
            # Add the column header
            if first_row:
                first_row = False
                row.append('Label')
                writer.writerow(row)
                continue
            
            # Add the label according to e_high and e_low
            label = 'Off'
            e_high = float(row[14]) * 60 * 0.2 - 24 * 4
            e_low = float(row[14]) * 60 * 0.18 - 24 * 3 - float(row[13])
            if e_high > 0 and e_high > e_low:
                label = 'High'
            elif e_low > 0:
                label = 'Low'
            row.append(label)
            writer.writerow(row)

In [3]:
def gen_prev_cols( rows ):
    cols = []
    for row in rows:
        cols = cols + row[4:len(row)-1] # skip the date and the mode
    return cols

In [4]:
def gen_n_days_headers( n ):
    default_headers = ['Pressure','Temp_max','Temp_mean','Temp_min','Temp_dew','Humidity','Cloud','Rainfall','Visi_redu','Sunshine','Radiation','Evaporation','Wind_dir','Wind_spd']
    headers = []
    for index in range(n):
        suffix = '_' + str(index+1) + 'd'
        headers = headers + [ header+suffix for header in  default_headers ]
    return headers    

In [5]:
num_of_prev_days = 5

with open('weather2017_4.csv', 'w', newline='') as outputfile:
    writer = csv.writer(outputfile, delimiter=',')
    with open('weather2017_3.csv') as inputfile:
        reader = csv.reader(inputfile)
        first_row = True
        headers = gen_n_days_headers(num_of_prev_days)
        prev_rows = [None] * num_of_prev_days
        
        for row in reader:
            # Add the column header
            if first_row:
                first_row = False
                row = row[0:2] + headers + [row[-1]]
                writer.writerow(row)
                continue
            
            # generate the output row
            if prev_rows[0] != None:
                output_row = row[0:2] + gen_prev_cols(prev_rows) + [row[-1]]
                writer.writerow(output_row)
                
            # Update prev_rows
            prev_rows.pop(0)
            prev_rows.append(row)